In [1]:
import random
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
from IPython.display import display, HTML
np.random.seed(42)
random.seed(42)
tqdm.pandas()

/var/folders/8p/5_r5jcmx2sl549t8tgwqvr3c0000gn/T/ipykernel_16750/1029000345.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
df = pd.read_csv("Data/interview_dataset.csv")
df["date on production"] = pd.to_datetime(df["date on production"], format = "%d/%m/%Y")
df["year"] = df["date on production"].dt.year
df["treatment company"] = [int(i[17:]) for i in df["treatment company"]]
df["operator"] = [int(i[8:]) for i in df["operator"]]
df = df.rename(columns = {"treatment company":"treatment_company"})
df = df.drop("date on production", axis = 1)
print(f"unique operators: {df.operator.nunique()}")
print(df.shape)
df.head()

unique operators: 36
(1000, 28)


,treatment_company,azimuth,md (ft),tvd (ft),operator,footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,...,s-velocity,youngs modulus,isip,breakdown pressure,pump rate,total number of stages,proppant volume,proppant fluid ratio,production,year
0,1,-32.280,19148,6443.000,1,11966.000,4368.463,6.330,12,1093,...,6950.440,30.820,4149.000,NaN,83,56,21568792.000,1.230,5614.948,2018
1,2,-19.800,15150,7602.000,2,6890.000,4714.992,1.280,4,0,...,7162.450,29.720,5776.000,NaN,102,33,9841307.000,1.470,2188.837,2014
2,3,-26.880,14950,5907.000,1,8793.000,798.921,2.030,6,3254,...,6976.930,30.990,4628.000,NaN,88,62,17116240.000,1.670,1450.033,2018
3,4,-49.100,11098,6538.000,1,4234.000,NaN,6.000,23,7470,...,6799.370,26.200,4582.000,NaN,100,11,3749559.000,0.770,1060.764,2012
4,5,5.560,10549,7024.000,3,2972.000,2967.563,11.870,9,3637,...,7046.910,31.180,4909.000,NaN,94,9,6690705.000,1.320,607.530,2012


# Train test validation split

For the split strategy I will split the dataset based on operator and year

Because some operators only appear once, all of them will be placed in the training set, ensuring the set has all the different operators.

In [4]:
oper_values = df.operator.value_counts().reset_index()
df_few = df[df.operator.isin(oper_values[oper_values["count"] < 20]["operator"])].reset_index(drop = True)
print(df_few.shape)
df = df[df.operator.isin(oper_values[oper_values["count"] > 20]["operator"])]
df = df.reset_index(drop = True)
df.operator.value_counts().tail()

(131, 28)


operator
25    46
20    45
2     40
26    28
9     23
Name: count, dtype: int64

In [5]:
feature_column = 'operator'

# Get the unique classes in the operator feature
classes = df[feature_column].unique()

subset1 = []
subset2 = []
ysubset1 = []
ysubset2 = []

# Iterate through the unique classes in operator
for class_ in classes:
    # Get samples belonging to the current class
    class_df = df[df[feature_column] == class_]
    class_X = class_df.drop("production", axis = 1)
    class_y = class_df["production"]
    
    # Split the samples into two subsets
    class_Xtrain, class_Xtest, class_ytrain, class_ytest= train_test_split(class_X,class_y, test_size=0.15, random_state=42)

    subset1.append(class_Xtrain)
    subset2.append(class_Xtest)
    ysubset1.append(class_ytrain)
    ysubset2.append(class_ytest)

X_train = pd.concat(subset1)
y_train = pd.concat(ysubset1)
X_test = pd.concat(subset2)
y_test = pd.concat(ysubset2)

print(f"train_frame shape: {X_train.shape}")
print(f"split_frame shape: {X_test.shape}")

train_frame shape: (735, 27)
split_frame shape: (134, 27)


In [6]:
X_few = df_few.drop("production", axis = 1)
y_few = df_few["production"]
# concat X_train with df_few
X_train = pd.concat([X_train,X_few]).reset_index(drop = True)
y_train = pd.concat([y_train,y_few]).reset_index(drop = True)
print(f"Train unique operators: {X_train.operator.nunique()}")
print(X_train.shape, y_train.shape)

Train unique operators: 36
(866, 27) (866,)


> Now train set have every possible value of operator

# Missing value Imputation

In [7]:
print(X_train.isnull().sum()[df.isnull().sum() > 0].sort_values())
null_cols = X_train.isnull().sum()[df.isnull().sum() > 0].sort_values()
null_cols = list(null_cols.index)
null_cols

tvd (ft)               11
youngs modulus         17
toc                    19
azimuth                46
isip                   65
porosity              101
proppant volume       115
well spacing          129
water saturation      505
breakdown pressure    641
dtype: int64


['tvd (ft)',
 'youngs modulus',
 'toc',
 'azimuth',
 'isip',
 'porosity',
 'proppant volume',
 'well spacing',
 'water saturation',
 'breakdown pressure']

> The imputation technique for the different features will be the mean value of year-operator

In [8]:
def impute_feature(data, column):
    oper_classes = data.operator.unique()
    # Iterate through the unique classes in year
    oper_list = []
    for oper in oper_classes:
        year_list = []
        oper_data = data[data.operator == oper]
        year_classes = oper_data.year.unique()
        for year in year_classes:
            # Get samples belonging to the current class
            class_df = oper_data[oper_data.year == year]
            mean_value = class_df[column].mean()
            class_df[column] = class_df[column].fillna(mean_value)
            year_list.append(class_df[column])
            
        years_series = pd.concat(year_list)
        years_series = years_series.fillna(years_series.mean())
        oper_list.append(years_series)

    column_series = pd.concat(oper_list)
    column_series = column_series.fillna(column_series.mean())
    
        
    return column_series.sort_index()

In [9]:
set_list = [X_train,X_test]
for set in set_list:
    for column in null_cols:
        set[column] = impute_feature(set, column)

print(f"Train Set nullcount: \n{X_train[null_cols].isnull().sum()}")
print(f"Test Set nullcount: \n{X_test[null_cols].isnull().sum()}")

/var/folders/8p/5_r5jcmx2sl549t8tgwqvr3c0000gn/T/ipykernel_16750/3547163216.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df[column] = class_df[column].fillna(mean_value)


Train Set nullcount: 
tvd (ft)              0
youngs modulus        0
toc                   0
azimuth               0
isip                  0
porosity              0
proppant volume       0
well spacing          0
water saturation      0
breakdown pressure    0
dtype: int64
Test Set nullcount: 
tvd (ft)              0
youngs modulus        0
toc                   0
azimuth               0
isip                  0
porosity              0
proppant volume       0
well spacing          0
water saturation      0
breakdown pressure    0
dtype: int64


# Save sets to disk

In [10]:
train_set = pd.concat([X_train,y_train], axis = 1)
test_set = pd.concat([X_test,y_test], axis = 1)
print(train_set.shape,test_set.shape)
train_set.head()

(866, 28) (134, 28)


,treatment_company,azimuth,md (ft),tvd (ft),operator,footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,...,s-velocity,youngs modulus,isip,breakdown pressure,pump rate,total number of stages,proppant volume,proppant fluid ratio,year,production
0,1,-14.430,14404,6437.000,1,7020.000,1152.576,34.400,9,0,...,6955.350,30.830,4491.000,6627.500,97,34,14222506.000,1.320,2016,3861.743
1,1,-16.320,10535,6035.000,1,4343.000,1852.010,6.960,3,0,...,7008.050,31.050,4176.000,7748.000,95,14,4437034.000,1.350,2013,512.233
2,8,-31.371,15111,6318.000,1,8372.000,2362.482,41.960,15,10042,...,6998.700,30.930,4650.647,6675.833,78,32,11923180.000,1.240,2013,1107.987
3,1,-48.480,12667,6714.000,1,5529.000,1052.818,10.640,3,2575,...,6877.280,30.660,4552.000,7625.250,99,36,10940004.000,1.230,2017,2943.454
4,1,-28.690,12606,6832.000,1,5121.000,902.851,46.200,6,0,...,7015.340,31.110,4939.000,7625.250,100,50,9865250.000,1.060,2017,2513.222


In [11]:
train_set.to_parquet("Data/processed/imputed_train.parquet")
test_set.to_parquet("Data/processed/imputed_test.parquet")